In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
from multiprocessing import Pool, Manager, cpu_count
import glob
import sys,os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()

In [2]:
with gezi.Timer('read train user_actions'):
  d = pd.read_csv('../input/user_action.csv')
  try:
    d1 = pd.read_csv('../input/v1/user_action.csv')
    d = pd.concat([d, d1])
  except Exception:
    pass

read train user_actions start
read train user_actions duration: 2.7867214679718018


In [3]:
with gezi.Timer('read test and merge'):
  dt = pd.read_csv('../input/test_a.csv')
  try:
    dt1a = pd.read_csv('../input/v1/test_a.csv')
    dt1b = pd.read_csv('../input/v1/test_b.csv')
    dt = pd.concat([dt, dt1a, dt1b])
  except Exception:
    dtb = pd.read_csv('../input/test_b.csv')
    dt = pd.concat([dt, dtb])

read test and merge start
read test and merge duration: 0.12278485298156738


In [4]:
dt['date_'] = 15

In [5]:
set(d.date_)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [6]:
dshow = pd.concat([d[['userid', 'feedid', 'date_']],
                  dt[['userid', 'feedid', 'date_']]])
dshow = dshow.sort_values(['date_'], ascending=True)

In [7]:
doc_vocab = gezi.Vocab('../input/doc_vocab.txt')

In [8]:
user_vocab = gezi.Vocab('../input/user_vocab.txt')

In [9]:
d = dshow.groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='docs')

  0%|          | 0/20000 [00:00<?, ?it/s]

In [10]:
d.head()

,userid,docs
0,8,"[71474, 56747, 105148, 25619, 48027, 83826, 40..."
1,12,"[69176, 102465, 85004, 36851, 106104, 80434, 3..."
2,13,"[15729, 60767, 23302, 31961, 52458, 30296, 796..."
3,25,"[49745, 79014, 2820, 49065, 69321, 104284, 846..."
4,37,"[10705, 32160, 60130, 2505, 18148, 49725, 4113..."


In [11]:
# with open('../input/show_dedges.txt', 'w') as f:
#   for row in tqdm(d.itertuples(), total=len(d)):
#     uid = user_vocab.id(row.userid) + doc_vocab.size()
#     for doc in row.docs:
#       print(uid, doc_vocab.id(doc), file=f)

In [20]:
ppp(1, 3, '2')

(1, 3, '2')
1 3 2


In [13]:
import ge2

In [14]:
import networkx as nx
from ge2.classify import read_node_label, Classifier
from ge2 import DeepWalk
G = nx.read_edgelist('../input/show_dedges.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph

In [15]:
model = DeepWalk(G,walk_length=10,num_walks=10,workers=cpu_count())#init model

[Parallel(n_jobs=96)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done   4 out of  97 | elapsed: 11.4min remaining: 265.8min
[Parallel(n_jobs=96)]: Done  97 out of  97 | elapsed: 268.6min finished


Learning embedding vectors...


TypeError: __init__() got an unexpected keyword argument 'size'

In [21]:
model.train(window_size=8,iter=10)# train model
embeddings = model.get_embeddings()# get embedding vectors

Learning embedding vectors...


TypeError: __init__() got an unexpected keyword argument 'size'

In [ ]:
# doc_emb = np.load('../input/15/doc_w2v_emb.npy')
doc_emb = np.load('../input/feed_pca_embeddings.npy')

In [ ]:
emb = np.zeros((user_vocab.size(), 128))

In [ ]:
for row in tqdm(d.itertuples(), total=len(d)):
  uid = user_vocab.id(row.userid)
  emb[uid] = np.mean([doc_emb[doc_vocab.id(x)] for x in row.docs])

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
emb

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00035974, -0.00035974, -0.00035974, ..., -0.00035974,
        -0.00035974, -0.00035974],
       ...,
       [ 0.00028186,  0.00028186,  0.00028186, ...,  0.00028186,
         0.00028186,  0.00028186],
       [ 0.00087649,  0.00087649,  0.00087649, ...,  0.00087649,
         0.00087649,  0.00087649],
       [ 0.0028019 ,  0.0028019 ,  0.0028019 , ...,  0.0028019 ,
         0.0028019 ,  0.0028019 ]])

In [ ]:
np.save('../input/15/user_mean_emb.npy', emb)

In [ ]:
np.save('../input/user_mean_emb.npy', emb)